This is a `bqplot` recreation of Mike Bostock's [Wealth of Nations](https://bost.ocks.org/mike/nations/). This was also done by [Gapminder](http://www.gapminder.org/world/#$majorMode=chart$is;shi=t;ly=2003;lb=f;il=t;fs=11;al=30;stl=t;st=t;nsl=t;se=t$wst;tts=C$ts;sp=5.59290322580644;ti=2013$zpv;v=0$inc_x;mmid=XCOORDS;iid=phAwcNAVuyj1jiMAkmq1iMg;by=ind$inc_y;mmid=YCOORDS;iid=phAwcNAVuyj2tPLxKvvnNPA;by=ind$inc_s;uniValue=8.21;iid=phAwcNAVuyj0XOoBL_n5tAQ;by=ind$inc_c;uniValue=255;gid=CATID0;by=grp$map_x;scale=log;dataMin=194;dataMax=96846$map_y;scale=lin;dataMin=23;dataMax=86$map_s;sma=49;smi=2.65$cd;bd=0$inds=;modified=60). It is originally based on a TED Talk by [Hans Rosling](http://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen).

In [ ]:
# Required imports
import pandas as pd
from bqplot import (LogScale, LinearScale, CategoricalColorScale, ColorAxis,
                    Axis, Scatter, CATEGORY10, Label, Figure)
from bqplot.default_tooltip import Tooltip
from ipywidgets import VBox, IntSlider, Button
from IPython.display import display
import os
import numpy as np
from time import sleep

In [ ]:
# The GUI starts with this year
initial_year = 1800

#### Cleaning and Formatting JSON Data

In [ ]:
data = pd.read_json(os.path.abspath('data_files/nations.json'))

In [ ]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [ ]:
data = clean_data(data)
data = extrap_data(data)

In [ ]:
income_min, income_max = np.min(data['income'].apply(np.min)), np.max(data['income'].apply(np.max))
life_exp_min, life_exp_max = np.min(data['lifeExpectancy'].apply(np.min)), np.max(data['lifeExpectancy'].apply(np.max))
pop_min, pop_max = np.min(data['population'].apply(np.min)), np.max(data['population'].apply(np.max))

In [ ]:
def get_data(year):
    year_index = year - 1800
    income = data['income'].apply(lambda x: x[year_index])
    life_exp = data['lifeExpectancy'].apply(lambda x: x[year_index])
    pop =  data['population'].apply(lambda x: x[year_index])
    return income, life_exp, pop

#### Creating the Tooltip to display the required fields

`bqplot`'s native `Tooltip` allows us to simply display the data fields we require on a mouse-interaction.

In [ ]:
tt = Tooltip(fields=['name', 'x', 'y'], labels=['Country Name', 'Income per Capita', 'Life Expectancy'])

#### Creating the Label to display the year

Staying true to the `d3` recreation of the talk, we place a `Label` widget in the bottom-right of the `Figure` (it inherits the `Figure` co-ordinates when no scale is passed to it). With `enable_move` set to `True`, the `Label` can be dragged around. 

In [ ]:
year_label = Label(x=0.85, y=0.1, font_size='52px', font_weight='bolder', color='orange',
                   text=str(initial_year), enable_move=True)

#### Defining Axes and Scales

The inherent skewness of the income data favors the use of a `LogScale`. Also, since the color coding by regions does not follow an ordering, we use the `CategoricalColorScale`.

In [ ]:
x_sc = LogScale(min=income_min, max=income_max)
y_sc = LinearScale(min=life_exp_min, max=life_exp_max)
c_sc = CategoricalColorScale(domain=data['region'].unique().tolist(), colors=CATEGORY10[:6])
size_sc = LinearScale(min=pop_min, max=pop_max)

In [ ]:
ax_y = Axis(label='Life Expectancy', scale=y_sc, orientation='vertical', side='left')
ax_x = Axis(label='Income per Capita', scale=x_sc)

#### Creating the Scatter Mark with the appropriate size and color parameters passed

To generate the appropriate graph, we need to pass the population of the country to the `size` attribute and its region to the `color` attribute.

In [ ]:
# Start with the first year's data
cap_income, life_exp, pop = get_data(initial_year)

In [ ]:
wealth_scat = Scatter(x=cap_income, y=life_exp, color=data['region'], size=pop,
                      names=data['name'], display_names=False,
                      scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': size_sc},
                      default_size=4112, tooltip=tt, animate=True, stroke='Black')

#### Creating the Figure

In [ ]:
fig = Figure(marks=[wealth_scat, year_label], axes=[ax_x, ax_y],
             title='Health and Wealth of Nations', background_style={'fill': 'White'},
             animation_duration=100)

#### Using a Slider to allow the user to change the year and a button for animation

Here we see how we can seamlessly integrate `bqplot` into the jupyter widget infrastructure. 

In [ ]:
year_slider = IntSlider(min=1800, max=2008, step=1, description='Year', value=initial_year)

In [ ]:
animate_button = Button(description='Play', background_color='MediumSeaGreen', color='Black', icon='fa-play')

On the slider value `callback` (a function that is triggered everytime the `value` of the slider is changed) we change the `x`, `y` and `size` co-ordinates of the `Scatter`. We also update the `text` of the `Label` to reflect the current year.

In [ ]:
def year_changed(new):
    wealth_scat.x, wealth_scat.y, wealth_scat.size = get_data(year_slider.value)
    year_label.text = str(year_slider.value)
    
year_slider.observe(year_changed, 'value')

#### Defining the callback for the button

When the `Button` is clicked, we trigger the animation by changing the slider value through all the years for which data is available. The change in the `Slider` value triggers a change in the plot.

In [ ]:
def button_clicked(value):
    animate_button.visible = False
    for i in range(1800, 2009, 1):
        year_slider.value = i
        sleep(0.05)
    animate_button.visible = True

animate_button.on_click(button_clicked)

#### Displaying the GUI

In [ ]:
display(VBox([animate_button, fig, year_slider]))